In [1]:
import os

# Execute this code only if in colab
if 'COLAB_GPU' in os.environ:
  print("Executing in Colab!")
  # Cloning GitHub repository
  !git clone https://github.com/fwitschel/QDMKM.git
  %cd QDMKM


Executing in Colab!
Cloning into 'QDMKM'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 77 (delta 13), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (77/77), 79.04 KiB | 2.47 MiB/s, done.
Resolving deltas: 100% (13/13), done.
/content/QDMKM


We install some libraries that we will need later

In [2]:
!pip install langchain langchain-community faiss-cpu langchain-anthropic groq rank_bm25

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.3/337.3 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


We read the input file into a so-called dataframe.

In [3]:
import pandas as pd
emails = pd.read_csv("/content/QDMKM/data/cases-emails.csv")
print(emails)

                 sender                 receiver  \
0     student42@MBFH.ch      my_msc_dean@MBFH.ch   
1     student21@MBFH.ch      my_msc_dean@MBFH.ch   
2     student84@MBFH.ch  former_msc_dean@MBFH.ch   
3    student168@MBFH.ch      my_msc_dean@MBFH.ch   
4    student336@MBFH.ch      my_msc_dean@MBFH.ch   
5    student672@MBFH.ch      my_msc_dean@MBFH.ch   
6     student72@MBFH.ch      my_msc_dean@MBFH.ch   
7     student67@MBFH.ch      my_msc_dean@MBFH.ch   
8      student7@MBFH.ch      my_msc_dean@MBFH.ch   
9    student745@MBFH.ch  former_msc_dean@MBFH.ch   
10    student45@MBFH.ch      my_msc_dean@MBFH.ch   
11    student74@MBFH.ch      my_msc_dean@MBFH.ch   
12     student5@MBFH.ch      my_msc_dean@MBFH.ch   
13   student666@MBFH.ch      my_msc_dean@MBFH.ch   
14   student888@MBFH.ch      my_msc_dean@MBFH.ch   
15  student4242@MBFH.ch         my_msc_dean@MBFH   

                                     subject  first_date  notice_weeks  \
0                        Re: late submiss

In the input file, each case is represented by one row. We create one Document object for each case that contains, as textual content (to be transformed and stored as embedding vectors) the subject of the initial email, followed by the entire text of the conversation. As metadata, we keep the email address of the sender and the notice with which the extension was requested. Later, it can be useful to have quick access to this metadata...

In [4]:
import datetime as dt
from langchain_core.documents import Document
docs = []
for index, row in emails.iterrows():
    sender = row['sender']
    subject = row['subject']
    text = subject + " " + row['all_text']
    notice = row['notice_weeks']
    year = dt.datetime.strptime(row['first_date'], '%Y-%m-%d').year
    document = Document(
        page_content=text,
        metadata={"source": sender,"notice":notice, "year":year},
        id = index
    )
    docs.append(document)

print(docs[0])

# later, when we combine ranks of documents, it will be useful to have a data structure
# that maps document indices to document objects:
doc_map = {}
for doc in docs:
  doc_map[doc.id] = doc

page_content='Re: late submission Dear Ms Smith, unfortunately, we cannot accept your request for deadline extension. Since your sickness occurred during a non-critical period of your thesis work and was comparatively short, there was enough time to resolve issues resulting from it. We are looking forward to receiving your thesis submission on June 21st. Best regards, The Dean. ---- Dear Prof. Dean, please find attached the certificate for my sickness. Hoping for a positive decision, best regards, Jane Smith. ---- Dear Ms Smith, could you please send us a medical certificate for your sick period. Please note that this does not imply that we will grant the extension, it is just a routine request. Thanks and best regards, The Dean. --- Dear Prof. Dean, I am writing to you to ask for a deadline extension of 1 week for my master thesis. In February, I had a really bad flu from which it took me two weeks to recover. I feel that I am still suffering from the consequences since my whole thesi

First, we want to index our emails for basic keyword retrieval. For this, we define a function that will do some pro-processing...

In [5]:
from nltk.stem.snowball import SnowballStemmer
import nltk

nltk.download('stopwords')

def preprocess(text):

    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # lowercase the tokens
    tokens = [token.lower() for token in tokens]

    # stemming
    stemmer = SnowballStemmer('german', ignore_stopwords=True)
    tokens = [stemmer.stem(token) for token in tokens]

    return tokens

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Then, we do the actual pre-processing on our documents. We store the pre-processed texts in a list, in the same order that they have in the document collection (thus, we can use the order to identify the metadata that belongs to a pre-processed text)

In [6]:
from rank_bm25 import BM25Okapi
import operator
import nltk
nltk.download('punkt_tab')

preprocessed_corpus = []
for doc in docs:
  doc_tokens = preprocess(doc.page_content)
  preprocessed_corpus.append(doc_tokens)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


We define the new case for which would like to retrieve and summarize similar historical cases. We also set the number (topk) of most similar cases to consider

In [7]:
topk = 3
query = "A student got a very late feedback regarding his MRTP that he wants to react to."
#query = "A student discovered that she was pregnant soon after starting the thesis proposal. Towards the end of her thesis, the pregancy became complicated and she had to take leave. A sickness certificate is available."
#query = "A student needs more time because he had to take over more responsibilities for a new project / mission. His employer assigned him as a project leader and he could not refuse it."

We build a keyword retrieval object and use it to run our query against all emails and to get a score for each email (hopefully indicating how relevant the email is for the new case)

In [8]:
bm25_retriever = BM25Okapi(preprocessed_corpus)
doc_scores = bm25_retriever.get_scores(preprocess(query))

# retrieve the documents (including metadata) corresponding to the topk highest scores
# by first inserting doc ids and corresponding scores into a dict and then sorting it by
# the values (=scores)
doc_scores_dict = {}
for i in range(len(doc_scores)):
  doc_scores_dict[i] = doc_scores[i]
sorted_scores = sorted(doc_scores_dict.items(), key=operator.itemgetter(1))

After getting the scores, we create a list where we insert pairs of documents (including their metadata) and their scores, only for the topk top-ranked documents...

In [9]:
bm25_results = []
for i in range(topk):
  (doc_id, score) = sorted_scores[len(docs)-i-1]
  cur_doc = docs[doc_id]
  bm25_results.append((cur_doc, score))

for result in bm25_results:
  print(result)

(Document(id='15', metadata={'source': 'student4242@MBFH.ch', 'notice': 0.0, 'year': 2023}, page_content='Re:MT Extension Dear Ms Grey, while this is very unfortunate, it is by no means certain that the consequences that you fear will arise. Please talk to your supervisor and clarify how to solve this. Without a feedback from her, I am not willing to grant an extension. Best regards, The Dean ---- Dear Dean, I am sorry to bother you about this, but I am really desperate: because my supervisor was sick for a long time in winter, I did not receive an MTRP assessment until now. Now, I got one and there is a point that I should correct that my supervisor sees as essential and I do not have the time to work it out now. I am afraid to get a very low grade if I do not! Could you grant me an extension by one or two weeks so that I can work this in? Thank you!'), np.float64(8.731743609582894))
(Document(id='9', metadata={'source': 'student745@MBFH.ch', 'notice': 6.0, 'year': 2015}, page_content

Now, we take an embeddings model and use it to create embeddings vectors for our email conversations. In the end, such an embedding vector is just a bunch of numbers...

In [10]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

#chunk_texts = list(map(lambda d: d.page_content, docs))
#embeddings = bge_embeddings.embed_documents(chunk_texts)
#print(embeddings[0])

/tmp/ipython-input-3020136791.py:6: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  bge_embeddings = HuggingFaceBgeEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

We store the embeddings in a vector store

In [11]:
from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(docs, bge_embeddings)

Then we retrieve the topk most similar cases using semantic search, i.e. comparison of embeddings.

In [12]:
semantic_results = db.similarity_search_with_score(query, k=topk)

for i in range(topk):
  print(semantic_results[i])

(Document(id='0', metadata={'source': 'student42@MBFH.ch', 'notice': 1.0, 'year': 2019}, page_content='Re: late submission Dear Ms Smith, unfortunately, we cannot accept your request for deadline extension. Since your sickness occurred during a non-critical period of your thesis work and was comparatively short, there was enough time to resolve issues resulting from it. We are looking forward to receiving your thesis submission on June 21st. Best regards, The Dean. ---- Dear Prof. Dean, please find attached the certificate for my sickness. Hoping for a positive decision, best regards, Jane Smith. ---- Dear Ms Smith, could you please send us a medical certificate for your sick period. Please note that this does not imply that we will grant the extension, it is just a routine request. Thanks and best regards, The Dean. --- Dear Prof. Dean, I am writing to you to ask for a deadline extension of 1 week for my master thesis. In February, I had a really bad flu from which it took me two week

Before we combine the results of keyword and semantic search, we also retrieve - for each document that any of the two searches has found - its age. We want to also factor that into the final ranking because old decisions should not impact new decisions as much as more recent ones.

In [13]:
# rank all documents that appear in either semantic search or keyword search results by age
ages = {}
for (doc,score) in bm25_results:
  ages[doc.id] = 2025 - doc.metadata['year']

for (doc,score) in semantic_results:
  ages[doc.id] = 2025 - doc.metadata['year']

sorted_ages = sorted(ages.items(), key=operator.itemgetter(1))
print(sorted_ages)

[('15', 2), ('6', 5), ('0', 6), ('13', 7), ('9', 10)]


Now, let's create a final ranking by using weighted reciprocal rang fusion (RRF)

In [14]:
k = 60
weights = {"bm25":0.4,"semantic":0.4,"age":0.2}

final_scores = {}
bm25_rank = 1
for (doc,score) in bm25_results:
  final_scores[doc.id] = weights["bm25"]/(k+bm25_rank)
  bm25_rank += 1

semantic_rank = 1
for (doc,score) in semantic_results:
  if doc.id in final_scores:
    final_scores[doc.id] += weights["semantic"]/(k+semantic_rank)
  else:
    final_scores[doc.id] = weights["semantic"]/(k+semantic_rank)
  semantic_rank += 1

age_rank = 1
for (doc_id,score) in sorted_ages:
  final_scores[doc_id] += weights["age"]/(k+age_rank)


sorted_final = sorted(final_scores.items(), key=operator.itemgetter(1))
final_results = []
for i in range(topk):
  (doc_id, score) = sorted_final[len(sorted_final)-i-1]
  cur_doc = doc_map[doc_id]
  final_results.append((cur_doc, score))

for (doc,score) in final_results:
  print(doc, score)

page_content='Re: help! Dear Mr Green, I am sorry to hear about these problems. However, your case is not one where we can grant an extension - it comes very late and the problem was known for a long time so that you could have taken appropriate countermeasures in due time (e.g. using a different method instead of a survey). We are looking forward to receiving your master thesis on Thursday. Best regards, The Dean ---- Dear Prof, I would like to ask for a deadline extension for my master thesis. I really got into trouble because of the poor response rate for the survey that I did in the beginning of my thesis. It meant that I had to do the survey again and I spent a lot of time convincing people to participate. All this was not planned... Would be great to hear from you, best regards, Greg' metadata={'source': 'student72@MBFH.ch', 'notice': 0.0, 'year': 2020} 0.01607950777702232
page_content='Re: late submission Dear Ms Smith, unfortunately, we cannot accept your request for deadline e

Here, we connect to an LLM at Groq. To make it work, please get yourself an API key for GROQ and store it as a key on the left side of this notebook...!

In [15]:
from groq import Groq
def llm(groq_client, prompt):
  chat_completion = groq_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama-3.3-70b-versatile",
  )

  return chat_completion.choices[0].message.content

In [16]:
from google.colab import userdata
groq_client = Groq(
    api_key=userdata.get('GROQ_API_KEY')
)

Here, you instruct the Large Language Model what to do:

* In the "system" part of the prompt, you explain the general task, including the
context (i.e. the retrieved information) that the system should rely on. You can pass the content of the retrieved emails by putting "{context}" into this part of the prompt
* In the "query" part of the prompt, you give instruction to make a decision about the new case (as introduced already above, before the retrieval)

In [17]:
context = '\n\n'.join(list(map(lambda c: c[0].page_content, final_results)))
prompt = f"""You are an assistant that helps a study dean to decide about students' request for extending the deadline of their master theses.
        The current case is described as follows: {query}.
        To decide about the current case, the following historical emails seem to be relevant: {context}. Please make a suggestion whether or not
        to grant the deadline extension, including a justification that is based on the given context! If possible, please include quotes from the historical emails"""
print(llm(groq_client, prompt))

Based on the given context, I suggest granting the deadline extension. The student's situation is unprecedented, as they received very late feedback regarding their MRTP, which they want to react to. This is not a case where the student had ample time to address the issue, as seen in previous cases.

For example, in the case of Mr. Green, the dean stated, "your case is not one where we can grant an extension - it comes very late and the problem was known for a long time so that you could have taken appropriate countermeasures in due time." In contrast, the current student received feedback at a very late stage, making it impossible for them to take countermeasures earlier.

Similarly, in the case of Ms. Smith, the dean noted that her sickness occurred during a non-critical period of her thesis work and was comparatively short, allowing her to resolve issues resulting from it. However, the current student's situation is different, as the late feedback has put them in a difficult positio